In [1]:
import pandas as pd
from tqdm import tqdm
import json
from openai import OpenAI

client = OpenAI()
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
yes = pd.read_csv('data/cohort_annotation_items.csv')
liste = yes['product_name'].dropna().unique()

In [17]:
food_df = pd.read_csv("data/foods-export-2025-04-10.csv")

print("Total foods:", food_df["food_name_en"].nunique())

print("Number of missing values in food_name_en:", food_df["food_name_en"].isna().sum())

food_df.head()

Total foods: 4213
Number of missing values in food_name_en: 0


,food_id,food_list_cname,food_name_en,food_name_fr,food_name_de,display_name_en,display_name_fr,display_name_de
0,0011c7e2-bd57-443b-98a7-5829a2a32ef3,ciqual_2020_07_07,"Mineral sparkling water (St-Diéry), bottled, s...","Eau minérale St-Diéry, embouteillée, gazeuse, ...",NaN,Strongly mineralized sparkling water (St-Diéry),Eau minérale gazeuse fortement minéralisée (St...,Stark mineralisiertes Mineralwasser (St-Diéry)
1,001663ae-31c8-40b1-a02e-3dc9020cd57a,ciqual_2020_07_07,"Sweetbread, lamb, raw","Ris, agneau, cru",NaN,Raw lamb sweetbread,Ris d'agneau cru,Rohes Lamm-Bries
2,0018c6f5-f40f-4958-aef1-97ec4a639537,sfcdb_v6_5,Black forest cake,Tourte fôret noire,Schwarzwälder Kirschtorte,Black forest cake,Forêt noire,Schwarzwälder Kirschtorte
3,00237a5f-22ce-4063-8eb6-1b6a04a0aecb,ciqual_2020_07_07,"Mineral still water (Luchon), bottled, lightly...","Eau minérale Luchon, embouteillée, non gazeuse...",NaN,Luchon mineral water (lightly mineralized),Eau minérale Luchon (légèrement minéralisée),Luchon Mineralwasser (leicht mineralisiert)
4,006e55bb-5f3a-49c3-a9af-642aae1f4f08,ciqual_2020_07_07,Fresh cream cheese cake,Gâteau au fromage blanc,NaN,Fresh cream cheese cake,Gâteau au fromage frais,Frischkäsekuchen


In [18]:
products_df = pd.read_csv("data/products.csv")

print("Total products:", products_df["product_id"].nunique())

print("Number of missing values in product_ids:", products_df["product_id"].isna().sum())

products_df.head()

Total products: 34
Number of missing values in product_ids: 0


,product_id,barcode,status,name_en,name_fr,name_de
0,02f7c7de-a88b-4bb1-b76a-08d61b86e954,7640151710071,complete,"Chiefs - Protein bar, crispy cookie",NaN,NaN
1,064b2b10-6020-4fd0-896e-e075fd53e9ef,7613356814189,complete,Glace vanille bâtonnet,Glace vanille bâtonnet,Glace vanille bâtonnet
2,088cf3cc-8ddf-41fc-902e-f41af49e3dcf,7610900217731,complete,Energy Milk Vanilla,Energy Milk Vanilla,Energy Milk Vanilla
3,0b9dd7e0-e8c6-49d6-8840-90a8002f312a,7627536694478,complete,High Protein Choco Drink,NaN,NaN
4,0c8a62da-48a6-4993-90fa-0d6a57081470,7627535701139,complete,Country Peanut choco chips,Country Peanut choco chips,Country Peanut Choco Chips


## Run openai

In [4]:
system_message = """
Categorize the given food item according to the Healthy Eating Index (HEI)-2020 components. Consider both adequacy and moderation aspects and determine whether the food’s composition falls into the ideal range for a high HEI score. For each component, return True if the food meets or exceeds the threshold for a maximum score (or, for moderation components, falls within the ideal low range), and False otherwise.

Follow these detailed guidelines:

### Adequacy Components:
1. **Total Fruits (5 points)**  
   - Includes 100% fruit juice.  
   - **Threshold:** Maximum score if ≥0.8 cup equivalents per 1,000 kcal; zero if no fruit is present.
2. **Whole Fruits (5 points)**  
   - Includes all fruit forms except juice.  
   - **Threshold:** Maximum score if ≥0.4 cup equivalents per 1,000 kcal; zero if absent.
3. **Total Vegetables (5 points)**  
   - Includes all vegetables, including beans, peas, and lentils.  
   - **Threshold:** Maximum score if ≥1.1 cup equivalents per 1,000 kcal; zero if none.
4. **Greens and Beans (5 points)**  
   - Includes dark green vegetables and legumes (beans, peas, lentils).  
   - **Threshold:** Maximum score if ≥0.2 cup equivalents per 1,000 kcal; zero if not present.
5. **Whole Grains (10 points)**  
   - Only whole grain products.  
   - **Threshold:** Maximum score if ≥1.5 ounce equivalents per 1,000 kcal; zero if absent.
6. **Dairy (10 points)**  
   - Includes all milk products (fluid milk, yogurt, cheese) and fortified soy beverages.  
   - **Threshold:** Maximum score if ≥1.3 cup equivalents per 1,000 kcal; zero if none.
7. **Total Protein Foods (5 points)**  
   - Includes all protein sources including legumes.  
   - **Threshold:** Maximum score if ≥2.5 ounce equivalents per 1,000 kcal; zero if absent.
8. **Seafood and Plant Proteins (5 points)**  
   - Includes seafood, nuts, seeds, soy products (except beverages), and legumes.  
   - **Threshold:** Maximum score if ≥0.8 ounce equivalents per 1,000 kcal; zero if none.

### Moderation Components:
9. **Refined Grains (10 points)**  
   - Products made with refined flours.  
   - **Threshold:** Maximum score if ≤1.8 ounce equivalents per 1,000 kcal and zero if ≥4.3 ounce equivalents per 1,000 kcal.
10. **Sodium (10 points)**  
    - **Threshold:** Maximum score if ≤1.1 grams per 1,000 kcal; zero if ≥2.0 grams per 1,000 kcal.
11. **Added Sugars (10 points)**  
    - **Threshold:** Maximum score if less than 6.5% of energy; zero if ≥26% of energy.
12. **Fatty Acids (10 points)**  
    - Based on the ratio of (polyunsaturated + monounsaturated fatty acids) to saturated fatty acids.  
    - **Threshold:** Maximum score if the ratio is ≥2.5; minimum score if the ratio is ≤1.2.

For each of these 12 components (using the keys given below), return a boolean that indicates if the food item's nutrient content is in the optimal range according to the HEI-2020 standards:

- total_fruits
- whole_fruits
- total_vegetables
- greens_and_beans
- whole_grains
- dairy
- protein_foods
- seafood_plant_proteins
- refined_grains
- sodium
- added_sugars
- fatty_acids

Be mindful that the food item may seem to fit multiple categories, in which case you should only generalise to the most likely (i.e., main) component, and not consider all or many possible components. Since I only have the entire weight available for that food item, and not individual weight contributions from each category, having many categories would overestimate the amount being sent to the HEI calculation.
"""

In [5]:
def categorize_food(food_name):
    """
    Categorize a food item according to HEI-2020 components

    Args:
        food_name (str): Name of the food to categorize

    Returns:
        dict: Dictionary with HEI food categories
    """
    # Initialize the OpenAI client
    # client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

    # Make the API call with structured output
    response = client.responses.create(
        model="gpt-4o-mini",
        # reasoning={"effort": "medium"},
        input=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": f"Categorize this food: {food_name}"},
        ],
        text={
            "format": {
                "type": "json_schema",
                "name": "food_category",
                "schema": {
                    "type": "object",
                    "properties": {
                        "total_fruits": {"type": "boolean"},
                        "whole_fruits": {"type": "boolean"},
                        "total_vegetables": {"type": "boolean"},
                        "greens_and_beans": {"type": "boolean"},
                        "whole_grains": {"type": "boolean"},
                        "dairy": {"type": "boolean"},
                        "protein_foods": {"type": "boolean"},
                        "seafood_plant_proteins": {"type": "boolean"},
                        "refined_grains": {"type": "boolean"},
                        "added_sugars": {"type": "boolean"},
                    },
                    "required": [
                        "total_fruits",
                        "whole_fruits",
                        "total_vegetables",
                        "greens_and_beans",
                        "whole_grains",
                        "dairy",
                        "protein_foods",
                        "seafood_plant_proteins",
                        "refined_grains",
                        "added_sugars",
                    ],
                    "additionalProperties": False,
                },
                "strict": True,
            }
        },
    )

    food_category = json.loads(response.output_text)
    food_category["food_name"] = food_name
    return food_category

In [6]:
food_name = (
    "Energy Milk Vanilla"  # "Honey, from flowers"  # "Zucchini piccata, prepared"
)

result = categorize_food(food_name)
print(result)

{'total_fruits': False, 'whole_fruits': False, 'total_vegetables': False, 'greens_and_beans': False, 'whole_grains': False, 'dairy': True, 'protein_foods': True, 'seafood_plant_proteins': False, 'refined_grains': False, 'added_sugars': True, 'food_name': 'Energy Milk Vanilla'}


In [7]:
import time

category_list = []

problem_foods = []

for e, food_name in tqdm(enumerate(liste)):
    try:
        res = categorize_food(food_name)
        category_list.append(res)
        if e % 100 == 0 and e != 0:
            pd.DataFrame(category_list).to_csv("data/new_products_category_list.csv", index=False)
            time.sleep(5)  # Pause for 5 seconds every 100 iterations
    except Exception as e:
        problem_foods.append(food_name)
        continue

368it [11:36,  1.89s/it]


In [27]:
import time
import pandas as pd
from tqdm import tqdm

category_list = []
problem_foods = []

products_df["best_name"] = products_df["name_en"].fillna(products_df["name_fr"]).fillna(products_df["name_de"])
products_df = products_df.dropna(subset=["best_name"])

unique_names = products_df["best_name"].drop_duplicates()

for e, food_name in tqdm(enumerate(unique_names)):
    try:
        res = categorize_food(food_name)
        category_list.append(res)
        if e % 100 == 0 and e != 0:
            pd.DataFrame(category_list).to_csv("data/products_category_list.csv", index=False)
            time.sleep(5)  # Pause for 5 seconds every 100 iterations
    except Exception as err:
        problem_foods.append(food_name)
        continue


0it [00:00, ?it/s]

34it [01:11,  2.09s/it]


In [8]:
products_df

NameError: name 'products_df' is not defined

In [29]:
pd.DataFrame(category_list).to_csv("data/products_category_list.csv")